### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [184]:
import pandas as pd
customers_solvency = pd.read_csv('/datasets/data.csv')
#Для первого знакомства с таблицей использовали методы: head(), info(), value_counts()
print(customers_solvency.head(10))
print()
print(customers_solvency.info())
print()
print(customers_solvency['education'].value_counts())
print()
print(customers_solvency['family_status'].value_counts())
print()
print(customers_solvency['gender'].value_counts())
print()
print(customers_solvency['income_type'].value_counts())
print()
print(customers_solvency['purpose'].value_counts())



   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод


При первоначальном ознакомлении с таблицей бросаются в глаза следующее:
1) В двух столбцах: days_employed и  total_income, количество данных, отличных от нуля - одинаково, однако меньше чем общее количество данных в остальных столбцах. 
2) В этих же двух столбцах данные представленны в виде вещественных чисел.
3) Артефакт в виде отрицательного количества дней трудового стажа
4) Мы видим разные форматы написания вариантов в столбце 'education': по разному пишется один и тот же вид образования
5) Разные форматы написания вариантов в столбце 'purpose'.
6) В одной из строк таблицы, в столбце 'gender' мы видим значение XNA.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [185]:
#Итак, мы обнаружили, что в некоторых ячейках столбцов days_employed и total_income отсутствуют числа
#Разберемся с отсутствующей информацией в этих столбцах. 
#Вызовем метод isnull для проверки данных столбцов на пропуски. Можно работать только с одним из этих
#двух столбцов, поскольку строки с пропусками одинаковы
print(customers_solvency[customers_solvency['total_income'].isna()].head())
print(customers_solvency[customers_solvency['total_income'].isna()].count())
#посмотрим для заемщиков каких видов дохода отсутствуют данные в этих двух столбцах 
print()
print(customers_solvency[customers_solvency['total_income'].isnull()]['income_type'].value_counts())


    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    пенсионер     0   
41        женат / замужем                 0      F  госслужащий     0   
55       гражданский брак                 1      F    пенсионер     1   

    total_income                           purpose  
12           NaN                   сыграть свадьбу  
26           NaN                       образование  
29   

In [186]:
#заменим NaN в столбце 'income_type' на медиану 
#сначала посчитаем медиану
try:
    income_median = customers_solvency.groupby('income_type').agg({'total_income': ['median']})
    print(income_median)
except:
    print('ошибка')

#сохраним медианы, необходимые для замены, в переменных
staffer_median = customers_solvency[customers_solvency['income_type'] == 'сотрудник']['total_income'].median()
partner_median = customers_solvency[customers_solvency['income_type'] == 'компаньон']['total_income'].median()
retiree_median = customers_solvency[customers_solvency['income_type'] == 'пенсионер']['total_income'].median()
official_median = customers_solvency[customers_solvency['income_type'] == 'госслужащий']['total_income'].median() 
owner_median = customers_solvency[customers_solvency['income_type'] == 'предприниматель']['total_income'].median()

# заменим NaN в строках. NaN наблюдаются в категориях сотрудник, компаньон, пенсионер, госслужащий, 
#предприниматель

customers_solvency.loc[(customers_solvency['income_type'] == 'сотрудник') & (customers_solvency['total_income'].isna()),'total_income'] = staffer_median
customers_solvency.loc[(customers_solvency['income_type'] == 'компаньон') & (customers_solvency['total_income'].isna()),'total_income'] = partner_median
customers_solvency.loc[(customers_solvency['income_type'] == 'пенсионер') & (customers_solvency['total_income'].isna()),'total_income'] = retiree_median
customers_solvency.loc[(customers_solvency['income_type'] == 'госслужащий') & (customers_solvency['total_income'].isna()),'total_income'] = official_median
customers_solvency.loc[(customers_solvency['income_type'] == 'предприниматель') & (customers_solvency['total_income'].isna()),'total_income'] = owner_median
print(customers_solvency.info())


                  total_income
                        median
income_type                   
безработный      131339.751676
в декрете         53829.130729
госслужащий      150447.935283
компаньон        172357.950966
пенсионер        118514.486412
предприниматель  499163.144947
сотрудник        142594.396847
студент           98201.625314
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory us

In [187]:
#Столбец с данными стажа нам в дальнейших расчетах не понадобиться.заменим NaN на нули:
customers_solvency['days_employed'] = customers_solvency['days_employed'].fillna(0)
print(customers_solvency.info())
#В столбце 'days_employed' мы видим отрицательные значения. 
#Определим общее количество строк с отрицательными значениями методом .count. 
#Этих строк 15906, то есть большая часть выборки. 
customers_solvency.loc[customers_solvency.loc[:,'days_employed'] < 0]['days_employed'].count()

#Предположим, что эта ошибка с отрицательными значениями не случайная, а закономерная. 
#Расчет трудового стажа обычно происходит с ипользование данных трудовой книжки: 
#как разность между текущей датой для занятого работника/датой окончания трудовой деятельности для 
#пенсионера и датой начала трудовой деятельности. 
#Предположим, что отрицательные значения появились как следствие перестановки уменьшаемого и 
#вычитаемого в формате UnixTime. 
#Чтобы эту ошибку исправить, можно просто умножить каждое отрицательное значение трудового стажа на -1. 
#И проверить методом .shape[0], все ли значения исправлены.
customers_solvency.loc[customers_solvency.loc[:,'days_employed'] < 0, ['days_employed']] = customers_solvency['days_employed']* (-1)
print(customers_solvency.head())
customers_solvency.loc[customers_solvency['days_employed'] < 0].shape[0]




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1  

0

In [166]:
#В этом же столбце 'days_employed' мы видим аномально высокие значения трудового стажа. 
#Отфильтруем все значения трудового стажа, которые превышают 17963 дня (количество дней, 
#прошедшее с 1 января 1970 по 8 марта 2019(последняя дата взята из теории курса в предположении 
#что выборка датируется именно этой датой)). Таких значений всего 3446. 
#Проверим для этой категории количество 'income_type'. 
#Мы видим, что высокие значения характерны в основном для заемщиков из категории "пенсионеры".
wrong_lines_content = customers_solvency.loc[customers_solvency.loc[:,'days_employed'] >= 17963]
print(wrong_lines_content['income_type'].value_counts())
print(wrong_lines_content.loc[wrong_lines_content.loc[:,'income_type'] == 'безработный'])
print(wrong_lines_content.loc[wrong_lines_content.loc[:,'income_type'] == 'сотрудник'])


пенсионер      3443
безработный       2
сотрудник         1
Name: income_type, dtype: int64
       children  days_employed  dob_years education  education_id  \
3133          1  337524.466835         31   среднее             1   
14798         0  395302.838654         45    Высшее             0   

          family_status  family_status_id gender  income_type  debt  \
3133    женат / замужем                 0      M  безработный     1   
14798  гражданский брак                 1      F  безработный     0   

        total_income                  purpose  
3133    59956.991984  покупка жилья для сдачи  
14798  202722.511368             ремонт жилью  
       children  days_employed  dob_years education  education_id  \
16335         1   18388.949901         61   среднее             1   

         family_status  family_status_id gender income_type  debt  \
16335  женат / замужем                 0      F   сотрудник     0   

        total_income                   purpose  
16335  186178.9

### Вывод

Мы обнаружили пропуски в двух столбцах 'days_employed' и 'total_income' в одних и тех же строках. Всего строк с пропущенными значениями - 2174 (определили с использованием метода count()). 
Это 10% от общего числа заемщиков Предполагаем, что значения NAN в этих столбцах не случайны:заемщики не предоставили информацию о стаже работы и доходах в анкете, либо у них нет официальных источников дохода либо эти данные потеряны. Столбец с данными стажа нам в дальнейших расчетах не понадобиться. Доход зависит от нескольких факторов: типа занятости, возраста, пола, наличия детей. В этом проекте мы заменим пропуски в столбце с ежемесячным доходом на медианы доходов соответствующих категорий типов дохода (сотрудники, пенсионеры и т.д.)
Отрицательные значения и аномально высокие значения стажа мы объясняем ошибкой в расчетах в формате unix time. Отрицательные значения мы заменили, аномально высокие значения оставили как есть, пропуски заменили на нули. Столбец со стажем в расчетах не используем. Его в принципе можно удалить, но мы оставили его, изменив так, чтобы не мешал в расчетах.


### Замена типа данных

In [167]:
#Заменим вещественный тип данных на целочисленный в столбцах 'days_employed' и 'total_income' методом .astype()
#customers_solvency['days_employed'] = customers_solvency['days_employed'].astype('int')
customers_solvency['total_income'] = customers_solvency['total_income'].astype('int')
customers_solvency['days_employed'] = customers_solvency['days_employed'].astype('int')
print(customers_solvency.info())
print(customers_solvency.head())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0   

Мы заменили вещественный тип данных на целочисленный в двух столбцах методом .astype()

### Обработка дубликатов

In [168]:
#В столбце 'education' находятся дублированные по смыслу категории, но по-разному форматированные. 
#Видимо заемщики заполняли анкету вручную, а не выбирая из предложенных вариантов.
print(customers_solvency['education'].value_counts())
print()

#Переведем в нижний регистр значения в этом столбце методом .str.lower().
customers_solvency['education'] = customers_solvency['education'].str.lower()
print()

#Тоже самое проделаем со столбцом family_purpose, поскольку там есть одно слово с большой буквой:
print(customers_solvency['family_status'].value_counts())
customers_solvency['family_status'] = customers_solvency['family_status'].str.lower()


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [169]:
#Применим метод .value_counts() для просмотра обновленных уникальных значений этих столбцов.
print(customers_solvency['education'].value_counts())
print(customers_solvency['family_status'].value_counts())


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [170]:
#Проверим всю таблицу на дубликаты:
print(customers_solvency.duplicated().sum())
#Удалим дубликаты
customers_solvency = customers_solvency.drop_duplicates().reset_index(drop = True)


71


In [171]:
#Проверим качество выполненных команд
print(customers_solvency.duplicated().sum())
print(customers_solvency.info())

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

В столбце 'education' находятся дублированные по смыслу категории, но по-разному форматированные. Видимо заемщики при заполнении анкеты впечатывали из предложенного перечня своими руками. Приведем в к единому формату значения в этом столбце методом .str.lower().Применили метод .value_counts() для просмотра обновленных уникальных значений этих столбцов. Применили метод.drop_duplicates для удаления дубликатов.


### Лемматизация

In [172]:
#выделим уникальные значения в столбце 'purpose' методом .unique и лемматизируем их:

from pymystem3 import Mystem
m = Mystem()
for i in range(len(customers_solvency['purpose'].unique())):
    lemmas = m.lemmatize(customers_solvency['purpose'][i])
    from collections import Counter
    print(Counter(lemmas))
    



Counter({'покупка': 1, ' ': 1, 'жилье': 1, '\n': 1})
Counter({'приобретение': 1, ' ': 1, 'автомобиль': 1, '\n': 1})
Counter({'покупка': 1, ' ': 1, 'жилье': 1, '\n': 1})
Counter({'дополнительный': 1, ' ': 1, 'образование': 1, '\n': 1})
Counter({'сыграть': 1, ' ': 1, 'свадьба': 1, '\n': 1})
Counter({'покупка': 1, ' ': 1, 'жилье': 1, '\n': 1})
Counter({' ': 2, 'операция': 1, 'с': 1, 'жилье': 1, '\n': 1})
Counter({'образование': 1, '\n': 1})
Counter({' ': 2, 'на': 1, 'проведение': 1, 'свадьба': 1, '\n': 1})
Counter({' ': 3, 'покупка': 1, 'жилье': 1, 'для': 1, 'семья': 1, '\n': 1})
Counter({'покупка': 1, ' ': 1, 'недвижимость': 1, '\n': 1})
Counter({' ': 2, 'покупка': 1, 'коммерческий': 1, 'недвижимость': 1, '\n': 1})
Counter({'сыграть': 1, ' ': 1, 'свадьба': 1, '\n': 1})
Counter({'приобретение': 1, ' ': 1, 'автомобиль': 1, '\n': 1})
Counter({' ': 2, 'покупка': 1, 'жилой': 1, 'недвижимость': 1, '\n': 1})
Counter({' ': 2, 'строительство': 1, 'собственный': 1, 'недвижимость': 1, '\n': 1})
Cou

### Вывод

Для приведения списка целей кредита к единообразию, мы лемматизировали исходный список для поиска ключевых слов. Ими оказались: недвижимость, автомобиль, образование, свадьба. В следующем разделе мы заменим данные списка с целями, на ключевые слова.


### Категоризация данных

In [173]:
#проведем категоризацию в столбце 'purpose' по полученным ключевым леммам
for i in range(len(customers_solvency['purpose'])):
    if ('недвижимост' in customers_solvency.loc[i, 'purpose']) or ('жиль' in customers_solvency.loc[i, 'purpose']):
        customers_solvency.loc[i, 'purpose'] = 'недвижимость'
    if 'авто' in customers_solvency.loc[i, 'purpose']:
        customers_solvency.loc[i, 'purpose'] = 'автомобиль'
    if 'образован' in customers_solvency.loc[i, 'purpose']:
        customers_solvency.loc[i, 'purpose'] = 'образование'
    if 'свадьб' in customers_solvency.loc[i, 'purpose']:
        customers_solvency.loc[i, 'purpose'] = 'свадьба'
        
#проверим полученный результат:
print(customers_solvency['purpose'].value_counts())
print(customers_solvency.head())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

        purp

In [174]:
##Категоризируем выборку по наличию или отсутствию детей:
def children_group(children):
    if children == 0:
        return 'бездетный'
    return 'с детьми'
customers_solvency['children_group'] = customers_solvency['children'].apply(children_group)
print(customers_solvency['children_group'].value_counts())
print(customers_solvency['children_group'].head(10))



бездетный    14091
с детьми      7363
Name: children_group, dtype: int64
0     с детьми
1     с детьми
2    бездетный
3     с детьми
4    бездетный
5    бездетный
6    бездетный
7    бездетный
8     с детьми
9    бездетный
Name: children_group, dtype: object


In [175]:
#категоризируем выборку по наличию или отсутствию партнера в две группы: есть партнер, нет партнера 
def family_status_group(family_status):
    if (family_status == 'женат / замужем') or ((family_status == 'гражданский брак')):
        return 'есть партнер'
    return 'нет партнера'
customers_solvency['family_status_group'] = customers_solvency['family_status'].apply(family_status_group)
print(customers_solvency['family_status_group'].value_counts())



есть партнер    16490
нет партнера     4964
Name: family_status_group, dtype: int64


In [176]:
# Категоризируем данные по уровню дохода. 
# сначала рассчитаем среднедушевой доход на семью, полагая, 
# что представленные заемщиками данные о доходе - это ежемесячный доход на семью
def family_total(row):
    family_status_group = row['family_status_group']
    children = row['children']
    family = 1
    for row in customers_solvency:
        if (family_status_group == 'нет партнера') & (children == 0):
            family = family
            return family
        if (family_status_group == 'нет партнера') & (children > 0):
            family = family + children
            return family
        if (family_status_group == 'есть партнер') & (children == 0):
            family = family + 1
            return family
        if (family_status_group == 'есть партнер') & (children > 0):
            family = family + 1 + children
            return family
#Проверим работоспособность функции
row_values = ['нет партнера', 2] 
row_columns = ['family_status_group', 'children']
row = pd.Series(data=row_values, index=row_columns) 
print (family_total(row))            
#создадим новый столбец family_group, где сохраним результаты работы нашей функции
customers_solvency['family_group'] = customers_solvency.apply(family_total, axis = 1)

# Создадим еще один столбец 'average_income', где сохраним значения среднедушевого дохода в семье:
customers_solvency['average_income'] = customers_solvency['total_income'] / customers_solvency['family_group']

print(customers_solvency.head())

# используем данные госкомстата за 2017 год для категоризации по уровню дохода на следующие: низкий доход - < 30000 ,
# средний доход <90000 , высокий доход > 90000
#создадим столбец 'income_per_person'
def income_per_person(average_income):
    if average_income <= 30000:
        return 'низкий доход'
    if average_income <= 90000:
        return 'средний доход'
    if average_income > 90000:
        return 'высокий доход'
      
customers_solvency['income_per_person'] = customers_solvency['average_income'].apply(income_per_person)   
print(customers_solvency['income_per_person'].value_counts())

3
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

        purpose children_group family_status_group  family_group  \
0  недвижимость       с детьми        есть партнер        

### Вывод

 Категоризировали данные в столбце целей кредита с помощью лемматизации для поиска ключевых слов и последующим использованием цикла для замены одинаковых по смыслу целей на ключевые слова. Работу с данными можно упростить, если банк оптимизирует и максимально упростит анкетирование своих заемщиков: заемщики лучще выбирать варианты из предложенного списка, а не заполнять самостоятельно


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [177]:
total_customers = len(customers_solvency)
print(total_customers)
#создадим сводную таблицу для дальнейшего анализа
debt_children = customers_solvency.groupby('children_group').agg({'debt':['sum'], 'children':['count']})

debt_children['debt_child_total'] = debt_children['debt']['sum'] / total_customers*100
debt_children['debt_child_categ'] = debt_children['debt']['sum'] / debt_children['children']['count'] *100

print()
print(debt_children)

21454

                debt children debt_child_total debt_child_categ
                 sum    count                                  
children_group                                                 
бездетный       1063    14091         4.954787         7.543822
с детьми         678     7363         3.160250         9.208203


### Вывод

Семьи с детьми являются более ответственными заемщиками, количество должников на 37% (1 - 3.2/5.0) меньше чем среди бездетных семей в общем количеств  заемщиков. Это и понятно - семьи с детьми более ответственно подходят к выбору целей кредита и планируют бюджет, кредит берут в случае неотложной необходимости(количество заемщиков с детьми почти в два раза меньше чем бездетных). Однако в своей категории доля невозвратов среди семей с детьми выше чем среди бездетных заемщиков. Что также объяснимо: в семьях с детьми больше вероятность форс мажорных обстоятельств. Например: если отца и мать сократили на работе, это сразу сильно бьет по семейному бюджету, поскольку на иждивении есть дети. И сохранившийся доход семьи используют на жизненно необходимые нужды в ущерб выплатам по кредитам. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [178]:
#создадим сводную таблицу для дальнейшего анализа
debt_spouse = customers_solvency.groupby('family_status_group').agg({'debt':['sum'], 'family_status':['count']})

debt_spouse['debt_spouse_total'] = debt_spouse['debt']['sum'] / total_customers*100
debt_spouse['debt_spouse_categ'] = debt_spouse['debt']['sum'] / debt_spouse['family_status']['count'] *100

print()
print(debt_spouse)


                     debt family_status debt_spouse_total debt_spouse_categ
                      sum         count                                    
family_status_group                                                        
есть партнер         1319         16490          6.148038          7.998787
нет партнера          422          4964          1.966999          8.501209


### Вывод

Мы видим, что в общем количестве доля неплательщиков среди заемщиков с партнером в три раза больше чем среди одиноких граждан. Но и заемщиков в браке в три раза больше чем одиноких заемщиков. Однако, если мы посмотрим на долю неплательщиков в своих категориях, то видим, что процент невозвратов практически одинаковый. То есть ответственность заемщиков не зависит от наличия или отсутствия партнера. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [179]:
try:
    by_income_type = customers_solvency.groupby('income_type').agg({'debt':['sum'], 'income_type': ['count']})
    print(by_income_type)
    
except:
    print('ошибка')
    

    

                 debt income_type
                  sum       count
income_type                      
безработный         1           2
в декрете           1           1
госслужащий        86        1457
компаньон         376        5078
пенсионер         216        3829
предприниматель     0           2
сотрудник        1061       11084
студент             0           1


In [180]:
#добавим столбец с долей просрочивших кредит в своей категории
#добавим столбец с долей просрочивших кредит в общем количестве заемщиков
by_income_type['debt_income_total']= by_income_type['debt']['sum'] / total_customers*100
by_income_type['debt_income_categ'] = by_income_type['debt']['sum'] / by_income_type['income_type']['count']*100 
print(by_income_type)



                 debt income_type debt_income_total debt_income_categ
                  sum       count                                    
income_type                                                          
безработный         1           2          0.004661         50.000000
в декрете           1           1          0.004661        100.000000
госслужащий        86        1457          0.400858          5.902539
компаньон         376        5078          1.752587          7.404490
пенсионер         216        3829          1.006805          5.641160
предприниматель     0           2          0.000000          0.000000
сотрудник        1061       11084          4.945465          9.572357
студент             0           1          0.000000          0.000000


### Вывод

Доходы распределяются ожидаемо по категорями дохода. Минимальное количество заемщиков в категориях декретниц, студентов, безработных и предпринимателей. Все логично: первые три категории обладают минимальным доходом. Отсутствие ожидаемого достаточно дохода в большинстве случаев является причиной отказа банка в кредите. Предприниматели реже обращаются в банки с целями кредита из нашей выборки. А если обращаются, то проблем с выплатами не имеют. Наибольшее количество заемщиков - в категории сотрудники со средним по всей выборке доходом. Но среди этих заемщиков и большее число неплательщиков, причем как во всей выборке, так и в своей категории. Наиболее ответственные заемщики   - пенсионеры. Имея постоянный доход ниже среднего по выборке, доля неплательщиков среди пенсионеров несколько ниже, чем в остальных группах с постянным доходом.
В целом можно отметить, что мы не наблюдаем линейной зависимости между возвратом кредита в срок и размером дохода. Поскольку на эту зависимость влияет комплекс факторов.
 

- Как разные цели кредита влияют на его возврат в срок?

In [181]:
#сгрупируем таблицу по целям кредита:
purpose_group = customers_solvency.groupby('purpose').agg({'debt':['sum'], 'purpose':['count']})

#сохраним в новые столбцы количество должников во всей выборке и в своей категории
purpose_group['debt_purpose_total']= purpose_group['debt']['sum'] / total_customers*100
purpose_group['debt_purpose_categ']= purpose_group['debt']['sum'] / purpose_group['purpose']['count']*100 
print(purpose_group)

   

             debt purpose debt_purpose_total debt_purpose_categ
              sum   count                                      
purpose                                                        
автомобиль    403    4306           1.878438           9.359034
недвижимость  782   10811           3.645008           7.233373
образование   370    4013           1.724620           9.220035
свадьба       186    2324           0.866971           8.003442


### Вывод

Основная цель кредита у заемщиков - недвижимость. При этом и доля невозвратов в общей выборке выше всего именной в этой категории. А вот в своей категории, доля неплательщиков самая низкая. Понятно почему: покупка недвижимости очень дорогостоящее, продуманное и жизненно важное мероприятие. Заемщики ответственно подходят к выплатам по кредиту. И только форсмажорные обстоятельства вынуждают заемщиков не платить по кредиту. Интересно, что наиболее импульсивные покупки, на которые берут кредит, это образование и автомобиль. Это "молодежные" категории и  очень похожи: количество кредитов на автомобиль и образование более чем в два раза меньше чем кредитов на жилье, а доля неплательщиков и в общей выборке и в своей категории также практически одинакова для обеих категорий. И процент невозврата этих видов кредитов в  самый большой в своих категориях. 


### Шаг 4. Общий вывод

1.Работу с данными можно упростить, если банк оптимизирует и максимально упростит анкетирование своих заемщиков: заемщикам лучше выбирать варианты из предложенного списка, а не заполнять самостоятельно во избежание появления ошибок.
2.Кредиты берут преимущественно семейные пары без детей. Эта категория граждан более уверена в завтрашнем дне и менее подвержена форсмажорным обстоятельствам, банки также лояльны к этой категории.
3. На вероятность возврата кредита влияет комплекс факторов. 
 - Наличие и количество детей, наличие супруга изменяют среднемесячный доход на человека в семье.  
 - Возраст влияет на цель кредита и вероятность кредита. Молодые люди более импульсивны и самонадеянны. Пенсионеры более осторожны в своих желаниях, к кредитам прибегают в крайних случаях
 - пол заемщика. У женщина в большинстве случаев, доход  ниже, чем у мужчин. 
 4. Самая популярная цель кредита - недвижимость, но процент неплательщиков в этой категории самый низкий. Самые импульсивные покупки, не обладающие жизненно необходимой важностью, это покупка автомобиля и получение образования. По этим видам кредита больше всего неплатежей.
 